In [24]:
import pytorchvideo.models as models
import torch
import torch.nn as nn
import pytorchvideo
import torch.nn.functional as F

from pytorch_lightning import LightningModule
import pytorch_lightning

In [25]:
%cd /workspace/Walk_Video_PyTorch/project/

/workspace/Walk_Video_PyTorch/project


In [26]:
resnet = models.create_resnet()

In [27]:
input_tensor = torch.zeros(2, 3, 8, 224, 224)
output = resnet(input_tensor)

In [28]:
output.shape

torch.Size([2, 400])

In [29]:
def make_walk_resnet():
    return pytorchvideo.models.resnet.create_resnet(
        input_channel = 3, 
        model_depth = 50, 
        model_num_class = 2, 
        norm = nn.BatchNorm3d,
        activation=nn.ReLU,
    )

In [30]:
class WalkVideoClassificationLightningModule(LightningModule):
    def __init__(self):
        super().__init__()
        self.model = make_walk_resnet()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])

        loss = F.cross_entropy(y_hat, batch["label"])

        self.log("train_loss", loss.item())

        return loss

    def validation_step(self, batch, batch_idx):
        y_hat = self.model(batch["video"])
        loss = F.cross_entropy(y_hat, batch["label"])
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-1)


In [31]:
class opt:
        _DATA_PATH = "/workspace/data/dataset/train"
        _CLIP_DURATION = 2 # Duration of sampled clip for each video
        _BATCH_SIZE = 10
        _NUM_WORKERS = 0  # Number of parallel processes fetching data

In [42]:
from dataloader.data_loader import WalkDataModule

def train():
    classification_module = WalkVideoClassificationLightningModule()

    data_module = WalkDataModule(opt)

    trainer = pytorch_lightning.Trainer(strategy='ddp')
    trainer.fit(classification_module, data_module)

In [43]:
train()

MisconfigurationException: `Trainer(strategy='ddp')` or `Trainer(accelerator='ddp')` is not compatible with an interactive environment. Run your code as a script, or choose one of the compatible strategies: Trainer(strategy=None|dp|tpu_spawn). In case you are spawning processes yourself, make sure to include the Trainer creation inside the worker function.

Bad pipe message: %s [b'" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome']
Bad pipe message: %s [b'v="102"\r\nsec-ch-ua-mobile: ?0\r\nse', b'ch-ua-platform: "Windows"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0;', b'in64; x64) AppleWebKit/537.36 (', b'TML, like Gecko) Chrome/102.0.5005.61 Safari/537.36\r\nAccept: text/html,app']
Bad pipe message: %s [b'cation/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchan', b';v=b3;q=0.9\r\nSec-Fetch-Site: none\r\nSec-Fetch-Mode: navigate\r\nSec-Fetch-User: ?1\r\nSec-Fetch-Dest: docum', b't\r\nAccept-Encoding: gzip, deflate, br\r\nAccept-Language: zh,zh-CN;q=0.9,zh-TW;q=0.8,ja;q=0.7,en;q=0.6']
Bad pipe message: %s [b'ol: max-age=0\r\nsec-ch-ua: " Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"\r\nsec-ch-ua-mobile: ?0\r']
Bad pipe message: %s [b'ec-ch-ua-', b'atform: "Windows"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Windows